In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
pathIn = "/home/priyanshjain/Downloads/Stride_Assignment/lane_input/"
pathOut = "lane_output/"

In [3]:
def video_to_frames(input_loc, output_loc):

    cap = cv2.VideoCapture(input_loc)
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        cv2.imwrite(output_loc + "%03d" % count + '.jpg', frame)
        count = count + 1

        if (count > (video_length-1)):
            cap.release()
            break

def frames_to_video(input_loc,output_loc):
    frame_array = []
    files = os.listdir(input_loc)  
    files.sort()

    for i,file_name in enumerate(files):
        img = cv2.imread(input_loc+file_name)
        height, width, layers = img.shape
        size = (width,height)
        frame_array.append(img)

    out = cv2.VideoWriter(output_loc,cv2.VideoWriter_fourcc(*'DIVX'), 14.0, size)

    for j in range(len(frame_array)):
        out.write(frame_array[j])
    out.release()

def load_input_frames(path):
    img = []
    images = os.listdir(path)
    images.sort()

    for i, image_name in enumerate(images):
        image = cv2.imread(path+image_name,1)
        img.append(image)
    
    return img

In [4]:
def canny_edge_detector(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    canny = cv2.Canny(blur, 50, 150)
    return canny  

def ROI_mask(image):
    height, width = image.shape
    polygons = np.array([ [(0, height), (0, round(height*0.85)), (round(width/2), round(height*0.55)), (width, round(height*0.85)), (width, height)] ]) 
    mask = np.zeros_like(image) 
    cv2.fillPoly(mask, polygons, 255) 
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

def get_coordinates (image, params):
     
    slope, intercept = params 
    y1 = image.shape[0]     
    y2 = int(y1 * (3/5)) 
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope) 
    
    return np.array([x1, y1, x2, y2])

def avg_lines(image, lines): 
    left = [] 
    right = [] 
    middle = []
    
    for line in lines: 
        x1, y1, x2, y2 = line.reshape(4)
        slope, y_intercept = np.polyfit((x1, x2), (y1, y2), 1)  

        if x1+x2/2 < 450 or x1+x2/2 > 1300:
            if slope < 0: 
                left.append((slope, y_intercept)) 
            else: 
                right.append((slope, y_intercept)) 
        if  -1.3 < slope < -0.7:
            middle.append((slope, y_intercept)) 
    
    left_avg = np.average(left, axis = 0) 
    right_avg = np.average(right, axis = 0) 
    middle_avg = np.average(middle, axis = 0)
    left_line = get_coordinates(image, left_avg) 
    right_line = get_coordinates(image, right_avg)
    middle_line = get_coordinates(image, middle_avg)
    
    return np.array([left_line, middle_line, right_line])

def draw_lines(image, lines, thickness): 
    line_image = np.zeros_like(image)
    color=[0, 0, 255]
    if lines is not None: 
        for x1, y1, x2, y2 in lines:
                    cv2.line(line_image, (x1, y1), (x2, y2), color, thickness)
    combined_image = cv2.addWeighted(image, 0.8, line_image, 1.0, 0.0)
    
    return combined_image

def lane(lines):
    slope = []
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        params = np.polyfit((x1, x2), (y1, y2), 1)  
        slope.append(params[0])

    if slope[0]*slope[1]>0:
        return "Car in Right Lane"
    elif slope[2]*slope[1]>0:
        return "Car in Left Lane"
    else:
        return "Car not on any Lane"

def laneFindingPipeLine(inputimage):
    canny_edges = canny_edge_detector(inputimage)
    cropped_image = ROI_mask(canny_edges)
    lines = cv2.HoughLinesP(cropped_image, rho=2, theta=np.pi/180, threshold=100, lines=np.array([]), minLineLength=40, maxLineGap=25 )
    averaged_lines = avg_lines (inputimage, lines)           
    combined_image = draw_lines(inputimage, averaged_lines, 5) 
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(combined_image,lane(averaged_lines),(500,100), font, 1, (255,0,0), 2, cv2.LINE_AA)
    return combined_image

In [5]:
def detect_lane_in_video():
    video_to_frames('Lane.mp4', pathIn)
    images = load_input_frames(pathIn)

    for i in range (0, len(images)):
        frame = laneFindingPipeLine(images[i])

    cv2.imwrite(pathOut+"%03d" % i+'.png', frame) 
    frames_to_video(pathOut, 'Lane_detection.mp4')

In [6]:
detect_lane_in_video()

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
